In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import pandas as pd


import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import timm


import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt


from dataloader import DigitDataset
from utils import reshape, drawimg, tensor_to_pltimg, multidrawimg
from utils_traintest import train, test
#Test

In [4]:
device = (torch.device('cuda') if torch.cuda.is_available()
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


### Data Loader

In [5]:
trainfile = pd.read_csv('digit-recognizer/train.csv')
trainimg = [(np.reshape(row[1].values[1:],(28,28)),row[1].values[0]) for row in trainfile.iterrows()]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,], [0.5,]),
    transforms.RandomAdjustSharpness(10)]
    
    )
batch_size=64
ds_train = DigitDataset('digit-recognizer/train.csv', transform)

dl_train = DataLoader(dataset=ds_train, batch_size=batch_size)


train_dataset, valid_dataset = train_test_split(ds_train, test_size=0.25, random_state=123, shuffle=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

imgs, labels = next(iter(dl_train))

imgs.shape, labels.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

### Model

In [7]:
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [8]:
model = timm.create_model('resnet10t')
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model = model.to('cuda')
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

5412520

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer,device=device)
    test(valid_loader, model, loss_fn, device=device)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 6.925791  [    0/31500]
loss: 5.878634  [ 6400/31500]
loss: 4.979755  [12800/31500]
loss: 4.028248  [19200/31500]
loss: 3.342632  [25600/31500]
Test Error: 
 Accuracy: 32.4%, Avg loss: 3.003568 

Epoch 2
-------------------------------
loss: 3.048427  [    0/31500]
loss: 1.884448  [ 6400/31500]
loss: 1.816658  [12800/31500]
loss: 1.790181  [19200/31500]
loss: 1.597830  [25600/31500]
Test Error: 
 Accuracy: 41.6%, Avg loss: 1.610983 

Epoch 3
-------------------------------
loss: 1.485727  [    0/31500]
loss: 1.368694  [ 6400/31500]
loss: 1.490656  [12800/31500]
loss: 1.508231  [19200/31500]
loss: 1.622020  [25600/31500]
Test Error: 
 Accuracy: 46.0%, Avg loss: 1.487613 

Epoch 4
-------------------------------
loss: 1.327301  [    0/31500]
loss: 1.412844  [ 6400/31500]
loss: 1.275488  [12800/31500]
loss: 1.355199  [19200/31500]
loss: 1.594514  [25600/31500]
Test Error: 
 Accuracy: 47.7%, Avg loss: 1.419005 

Epoch 5
------------------------

In [11]:
batch_size = 64
loss_fn = nn.CrossEntropyLoss()
ds_test = DigitDataset('digit-recognizer/test.csv', transforms.ToTensor())
test_loader = DataLoader(dataset=ds_test, batch_size=batch_size, shuffle=False)

model = timm.create_model('resnet10t')
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model = model.to('cuda')
model.load_state_dict(torch.load("model.pth"))
test(test_loader, model, loss_fn, device=device, valid=False)

AssertionError: Torch not compiled with CUDA enabled